## Imports and data-reading

In [1]:
import pandas as pd
# !pip install cvxopt
import numpy as np
import matplotlib.pyplot as plt
from cvxopt import matrix, solvers
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.svm import SVC
from sklearn import datasets

## Useful Functions for any part

### For SVM/SVC plotting

In [2]:
def plot_it_all_rbf(train_X, train_y, test_X, test_y, reg_pam, ker_coeff_pam):

    ######################################### 3D plot ########################################
    Z = np.ones((len(reg_pam),len(ker_coeff_pam)))
    W = np.ones((len(reg_pam),len(ker_coeff_pam)))
    X, Y = np.meshgrid(reg_pam, ker_coeff_pam)


    for idxc, c in enumerate(reg_pam):
        for idxg, g in enumerate(ker_coeff_pam):
            ppl = [('scaler', StandardScaler()), ('SVM', svm.SVC(kernel='poly', C = c, gamma = g))]
            pipeline = Pipeline(ppl) 
            mod = pipeline
#             mod = svm.SVC(kernel='poly', C = c, gamma = g)
            mod.fit(train_X, np.ravel(train_y, order='C'))
            Z[idxc][idxg] = mod.score(train_X, train_y)
            W[idxc][idxg] = mod.score(test_X, test_y)

    # print(X, Y, Z, W, X.shape, Y.shape, Z.shape, W.shape)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    #ax.plot_wireframe(np.log10(X), np.log10(Y), Z, rstride=1, cstride=1, label='train', color='blue' )
    #ax.plot_wireframe(np.log10(X), np.log10(Y), W, rstride=1, cstride=1, label='test', color='red')
    #ax.plot_surface(np.log10(X), np.log10(Y), Z, rstride=1, cstride=1, label='train', cmap='viridis', edgecolor='none' )
    #ax.plot_surface(np.log10(X), np.log10(Y), W, rstride=1, cstride=1, label='test', cmap='viridis', edgecolor='none')
    ax.contour(np.log10(X), np.log10(Y), Z, label='train', cmap='viridis')
    ax.contour(np.log10(X), np.log10(Y), W, label='test', cmap='viridis')
    ax.colorbar()
    ax.set_title('Poly kernel: C and Gamma Vs Accuracy')
    ax.set_xlabel('log(C)')
    ax.set_ylabel('log(Gamma)')

    plt.legend()
    plt.show()


    ######################################## My conventional plots ########################################
    for idxc, c in enumerate(reg_pam):
        E_score_train = []
        E_score_test = []

        for idxg, g in enumerate(ker_coeff_pam):
            ppl = [('scaler', StandardScaler()), ('SVM', svm.SVC(kernel='poly', C = c, gamma = g))]
            pipeline = Pipeline(ppl) 
            mod = pipeline
            #mod = svm.SVC(kernel='poly', C = c, gamma = g)
            mod.fit(train_X, np.ravel(train_y, order='C'))
            E_score_train.append(mod.score(train_X, train_y))
            E_score_test.append(mod.score(test_X, test_y))

        plt.plot(ker_coeff_pam, E_score_test, "r-", label="Test")
        plt.plot(ker_coeff_pam, E_score_train, "b-", label="Train")
        plt.xscale('log')
        plt.xlabel(r'$\Gamma$')
        plt.ylabel('Score')
        plt.legend()
        plt.title('Score for C=%s'%(c))
        plt.show()


        
def plot_it_all_linear(train_X, train_y, test_X, test_y, reg_pam):

    ######################################## My conventional plots ########################################
    E_score_train = []
    E_score_test = []

    for idxc, c in enumerate(reg_pam):    
        ppl = [('scaler', StandardScaler()), ('SVM', svm.SVC(kernel='linear', C = c))]
        pipeline = Pipeline(ppl) 
        mod = pipeline
#         mod = svm.SVC(kernel='poly', C = c, gamma = g)
        mod.fit(train_X, np.ravel(train_y, order='C'))
        E_score_train.append(mod.score(train_X, train_y))
        E_score_test.append(mod.score(test_X, test_y))

    plt.plot(reg_pam, E_score_test, "r-", label="Test")
    plt.plot(reg_pam, E_score_train, "b-", label="Train")
    plt.title("Score Vs C for LIBSVM")
    plt.xscale('log')
    plt.xlabel('C')
    plt.ylabel('Score')
    plt.legend()
#     plt.title('Score for C=%s'%(c))
    plt.show()

        
        
        
######################################## PLOTS FOR GRID SEARCH ########################################

def plot_search_results(grid):
    # Reference: https://stackoverflow.com/questions/37161563/how-to-graph-grid-scores-from-gridsearchcv
    """
    Params: 
        grid: A trained GridSearchCV object.
    """
    ## Results from grid search
    results = grid.cv_results_
    means_test = results['mean_test_score']
    stds_test = results['std_test_score']
    means_train = results['mean_train_score']
    stds_train = results['std_train_score']

    ## Getting indexes of values per hyper-parameter
    masks=[]
    masks_names= list(grid.best_params_.keys())
    for p_k, p_v in grid.best_params_.items():
        masks.append(list(results['param_'+p_k].data==p_v))

    params=grid.param_grid

    ## Ploting results
    fig, ax = plt.subplots(1,len(params),sharex='none', sharey=False,figsize=(len(params)*10,len(params)*2.5))
    fig.suptitle('Score per parameter')
    fig.text(0.085, 0.5, 'MEAN SCORE', va='center', rotation='vertical')
    pram_preformace_in_best = {}
    for i, p in enumerate(masks_names):
        m = np.stack(masks[:i] + masks[i+1:])
#         m=np.array(masks[:i])
        pram_preformace_in_best
        best_parms_mask = m.all(axis=0)
        best_index = np.where(best_parms_mask)[0]
        x = np.array(params[p])
        y_1 = np.array(means_test[best_index])
        e_1 = np.array(stds_test[best_index])
        y_2 = np.array(means_train[best_index])
        e_2 = np.array(stds_train[best_index])
        if (x[-1]>=x[-2]*9):
            ax[i].set_xscale('log')
        ax[i].errorbar(x, y_1, e_1, linestyle='--', marker='o', label='test')
        ax[i].errorbar(x, y_2, e_2, linestyle='-', marker='^',label='train' )
        ax[i].set_xlabel(p.upper())
        ax[i].grid(True)
        ax[i].legend()

    #plt.legend()
    plt.show()


    
def plot_search_results_linear(grid, reg_pam):
    results = grid.cv_results_
    means_test = results['mean_test_score']
    stds_test = results['std_test_score']
    means_train = results['mean_train_score']
    stds_train = results['std_train_score']

#     print("")
    # print('mean_test_score: ', means_test)
    # print('std_test_score: ', stds_test)
    # print('mean_train_score: ', means_train)
    # print('std_train_score: ', stds_train)
    x = reg_pam
    y_1 = np.array(means_test)
    e_1 = np.array(stds_test)
    y_2 = np.array(means_train)
    e_2 = np.array(stds_train)
    plt.errorbar(x, y_1, e_1, linestyle='--', marker='o', label='test')
    plt.errorbar(x, y_2, e_2, linestyle='-', marker='^',label='train' )
    plt.grid(True)
    plt.xscale('log')
    plt.title('Score Vs C for CVXOPT')
    plt.legend(loc='lower right')
    plt.ylabel('Score')
    plt.xlabel('C')
    plt.show()

### For CVXOPT

In [3]:
# Reference: https://courses.csail.mit.edu/6.867/wiki/images/a/a7/Qp-cvxopt.pdf
# Reference: https://www.robots.ox.ac.uk/~az/lectures/ml/lect3.pdf

def cvx_try(X, y, X_test, y_test, which, C=1.0, gamma=0.04, coef0= 0.0, degree=3, threshold=1e-4):
    
    # Linear: C
    # Poly: C, Gamma, Degree, Coef0
    # Sigmoid: C, Gamma, Coef0
    # RBF: C, gamma
    
    def makeHP(X1, X2, y_temp, which):
        H = np.dot(X1, X2.T)
        if (which=='linear'):
            P = matrix(y_temp.dot(y_temp.T)*H)
        elif (which=='poly'):
            H = (gamma*H+coef0)**degree
            P = matrix(y_temp.dot(y_temp.T)*H)
        elif (which=='sigmoid'):
            H = np.tanh(gamma*H+coef0)
            P = matrix(y_temp.dot(y_temp.T)*H)
        elif (which=='rbf'):
            H1 = np.diag(X1.dot(X1.T)).reshape(-1, 1)*np.ones((1, X2.shape[0]))
            H2 = np.diag(X2.dot(X2.T)).reshape(1, -1)*np.ones((X1.shape[0],1))
            H = 2*H-H1-H2
            H = np.exp(gamma*H)
            P = matrix(y_temp.dot(y_temp.T)*H)
    
        return H,P
    
    y_temp = y.reshape(-1, 1)*1.
    H,P = makeHP(X, X, y_temp, which)
    q = matrix(-np.ones((X.shape[0], 1)))
    A = matrix(y_temp.reshape(1, -1))
    b = matrix(np.zeros(1))
    G = matrix(np.r_[-1*(np.eye(X.shape[0])), np.eye(X.shape[0])])
    h = matrix(np.r_[np.zeros(X.shape[0]), np.ones(X.shape[0])*C])
    #print(repr(q),repr(A),repr(b),repr(G),repr(h))
    solvers.options['show_progress'] = False
    sol = solvers.qp(P,q,G,h,A,b)
    lambs, _ = np.array(sol['x']), np.array(sol['primal objective'])
    
    
    ########## GET SUPPORT VECTORS ################
    idx = np.where(lambs > threshold)[0] # Indices of support vectors
    #Extract support vectors
    sX = X[idx,:]
    sy = y[idx]
    lambs = lambs[idx]
    b = np.sum(sy)
    for j in idx:
        b = b - np.sum(lambs*sy*(H[j, idx].reshape(-1, 1)))
    b /= idx.shape[0]
    
    
    
    ######### PREDICT ###########
    ynew = np.zeros((X_test.shape[0],))
    Htemp,_ = makeHP(sX, test_X, ynew, which)
    rightcnt = 0.
#     print(ynew.shape, Htemp.shape)
    for i in range(ynew.shape[0]):
        ynew[i] = np.sum(lambs*sy*Htemp[:,i].reshape(-1,1))+b
        if (ynew[i]*y_test[i]>0):
            rightcnt+=1.
    y_pred = np.sign(ynew)
    score = rightcnt*100.0/ynew.shape[0]
    
    
    
    return lambs, sX, sy, y_pred, score

# Part 1A

## CVXOPT

In [ ]:
# # cvx_try(X, y, X_test, y_test, which, C=1.0, gamma=0.04, coef0= 0.0, degree=3, threshold=1e-4):
# legr, suppX, suppy, test_pred, acc = cvx_try(train_X, train_y, test_X, test_y, which='poly', C=0.01, gamma=1000, degree=1)


# # threshold = 1e-4
# # ############ https://xavierbourretsicotte.github.io/SVM_implementation.html #########
# # #w parameter in vectorized form
# # w = ((train_y * idk).T @ train_X).reshape(-1,1)
# # #Selecting the set of indices S corresponding to non zero parameters
# # S = (idk > threshold).flatten()
# # #Computing b
# # b = train_y[S] - np.dot(train_X[S], w)
# # #Display results
# # # print('Alphas = ',idk[idk > threshold])
# # print('w = ', w.flatten())
# # print('b = ', b[0])

# # suppV = (suppX, suppy)
# # print("Support vectors: ", suppV)
# print("Support vectors: ", sorted(suppX, key=lambda x: x[0]))
# print("CVX score: ", acc)

In [5]:
import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning)
# warnings.filterwarnings("ignore", category=SettingWithCopyWarning)
# warnings.resetwarnings()
warnings.filterwarnings("ignore")

# DEATH BY PIPELINE

## Linear

In [ ]:
file_name = '2019EE10143.csv'
split_frac = 0.8
random_state = 69420
low_reg, high_reg = -5,5
low_cof, high_cof = -3,3
low_deg, high_deg = 1, 6

reg_pam = np.logspace(low_reg, high_reg, num=1+high_reg-low_reg)
ker_coeff_pam = np.logspace(low_cof, high_cof, num=1+high_cof-low_cof)
deg_pam = np.linspace(low_deg, high_deg, 1+low_deg+high_deg)


df = pd.read_csv(file_name, header=None)
pairs = [(0,1),(4,6),(8,9)]
features = [10, 25]

for (lab1, lab2) in pairs:
    for num_ft in features:
        print("#################################################################################")
        print("Labels: ",lab1, ", ", lab2)
        print("Number of features: ", num_ft)
        print("#################################################################################")

        #CONVERT TO USE
        df_temp = df.loc[df[25].isin([lab1, lab2])]
        #print(len(df_temp))
        df_temp.iloc[df_temp[25] == lab1, 25] = -1
        df_temp.iloc[df_temp[25] == lab2, 25] = 1
        df_temp = df_temp.sample(frac=1., random_state=random_state)
        
        # SPLIT IN TRAIN AND TEST
        train_df = df_temp[:int(split_frac*len(df_temp))]
        test_df = df_temp[int(split_frac*len(df_temp)):]
        
        # SPLIT BY FEATURES
        X_train_temp = train_df.loc[:, [i for i in range(num_ft)]]
        y_train_temp = train_df.loc[:, [25]]
        X_test_temp = test_df.loc[:, [i for i in range(num_ft)]]
        y_test_temp = test_df.loc[:, [25]]



        train_X = np.array(X_train_temp.values)
        train_y = np.array(y_train_temp.values)
        test_X = np.array(X_test_temp.values)
        test_y = np.array(y_test_temp.values)

        print ("Number of training examples: ", train_X.shape, train_y.shape)
        print ("Number of test examples: ", test_X.shape, test_y.shape)


        # SVM STUFF & GRID SEARCH CV
        print("--------------------------LIBSVM-----------------------------")
        ppl = [('scaler', StandardScaler()), ('SVM', svm.SVC(kernel='linear'))]
        pipeline = Pipeline(ppl) 
        parameters = {'SVM__C':reg_pam} #Linear
        #parameters = {'SVM__C':reg_pam, 'SVM__degree':deg_pam, 'SVM__gamma':ker_coeff_pam} #Poly
        # parameters = {'SVM__C':reg_pam, 'SVM__degree':deg_pam, 'SVM__gamma':ker_coeff_pam}
        grid = GridSearchCV(pipeline, param_grid=parameters, cv=5, return_train_score=True)

        grid.fit(train_X, np.ravel(train_y, order='C'))
        #print(grid.score(train_X, train_y))
        #print(grid.score(test_X, test_y))
        bestpar = grid.best_params_
        print("The Best parameters according to grid search are: ", bestpar)

        mod = svm.SVC(kernel='linear', C = bestpar['SVM__C']) #Linear
        #mod = svm.SVC(kernel='poly', C = bestpar['SVM__C'], gamma = bestpar['SVM__gamma'], degree= bestpar['SVM__degree']) #Poly
        
        mod.fit(train_X, np.ravel(train_y, order='C'))
        print("Training score for LIBSVM with best parameters: ", 100*mod.score(train_X, train_y), "%")
        print("Test score for LIBSVM with best parameters: ", 100*mod.score(test_X, test_y), "%")
        #print("Support vectors as returned by LIBSVM: ", sorted(mod.support_vectors_, key=lambda x: x[0]))
        ab,a_ind,b_ind=np.intersect1d(train_X, mod.support_vectors_, return_indices=True)
        boi = []
        a_ind = sorted(a_ind)
        for i in range(0, len(a_ind), num_ft):
            boi.append(a_ind[i]//num_ft)
        #print("Indices of support vectors as returned by LIBSVM: ", [temp_train_X.index(x) for x in mod.support_vectors_])
        print("Indices of support vectors as returned by LIBSVM: ", boi)
        print()
        print()
        print()
        
        # CVXOPT STUFF
        print("--------------------------CVXOPT-----------------------------")
        legr, suppX, suppy, test_pred, acc = cvx_try(train_X, train_y, test_X, test_y, which='linear', C=bestpar['SVM__C']) #Linear
        #legr, suppX, suppy, test_pred, acc = cvx_try(train_X, train_y, test_X, test_y, which='poly', C=bestpar['SVM__C'],  #Poly pt1
        #                                             gamma = bestpar['SVM__gamma'], degree= bestpar['SVM__degree']) #Poly pt2

        print("Test score for CVXOPT with best parameters: ", acc, "%")
        ab,a_ind,b_ind=np.intersect1d(train_X, suppX, return_indices=True)
        boi = []
        a_ind = sorted(a_ind)
        for i in range(0, len(a_ind), num_ft):
            boi.append(a_ind[i]//num_ft)
        print("Indices of support vectors as returned by CVXOPT: ", boi)
        #print("Support vectors as returned by CVXOPT: ", sorted(suppX, key=lambda x: x[0]))

        
        
        # PLOTTINGS
        plot_it_all_linear(train_X, train_y, test_X, test_y, reg_pam) #Linear. Nothing for anything else. Can use other one for RBF
        plot_search_results_linear(grid, reg_pam) #Linear
        #plot_search_results(grid) #For non-linear or any with multiple parameters
        
        print()
        print()
        print()
        print()
        print()
        print()

## Poly

In [ ]:
file_name = '2019EE10143.csv'
split_frac = 0.8
random_state = 69420
low_reg, high_reg = -3,3
low_cof, high_cof = -3,3
low_deg, high_deg = 1, 6

reg_pam = np.logspace(low_reg, high_reg, num=1+high_reg-low_reg)
ker_coeff_pam = np.logspace(low_cof, high_cof, num=1+high_cof-low_cof)
deg_pam = np.linspace(low_deg, high_deg, 1+high_deg-low_deg)


df = pd.read_csv(file_name, header=None)
pairs = [(0,1),(4,6),(8,9)]
# pairs = [(8,9)]
features = [10, 25]

for (lab1, lab2) in pairs:
    for num_ft in features:
        print("#################################################################################")
        print("Labels:",lab1, ",", lab2)
        print("Number of features:", num_ft)
        print("#################################################################################")

        #CONVERT TO USE
        df_temp = df.loc[df[25].isin([lab1, lab2])]
        #print(len(df_temp))
        df_temp.iloc[df_temp[25] == lab1, 25] = -1
        df_temp.iloc[df_temp[25] == lab2, 25] = 1
        df_temp = df_temp.sample(frac=1., random_state=random_state)
        
        # SPLIT IN TRAIN AND TEST
        train_df = df_temp[:int(split_frac*len(df_temp))]
        test_df = df_temp[int(split_frac*len(df_temp)):]
        
        # SPLIT BY FEATURES
        X_train_temp = train_df.loc[:, [i for i in range(num_ft)]]
        y_train_temp = train_df.loc[:, [25]]
        X_test_temp = test_df.loc[:, [i for i in range(num_ft)]]
        y_test_temp = test_df.loc[:, [25]]



        train_X = np.array(X_train_temp.values)
        train_y = np.array(y_train_temp.values)
        test_X = np.array(X_test_temp.values)
        test_y = np.array(y_test_temp.values)

        print ("Number of training examples:", train_X.shape, train_y.shape)
        print ("Number of test examples:", test_X.shape, test_y.shape)


        # SVM STUFF & GRID SEARCH CV
        print("--------------------------LIBSVM-----------------------------")
        ppl = [('scaler', StandardScaler()), ('SVM', svm.SVC(kernel='poly'))]
        pipeline = Pipeline(ppl) 
        #parameters = {'SVM__C':reg_pam} #Linear
        parameters = {'SVM__C':reg_pam, 'SVM__degree':deg_pam, 'SVM__gamma':ker_coeff_pam} #Poly
        # parameters = {'SVM__C':reg_pam, 'SVM__degree':deg_pam, 'SVM__gamma':ker_coeff_pam}
        grid = GridSearchCV(pipeline, param_grid=parameters, cv=5, return_train_score=True, verbose=0)

        grid.fit(train_X, np.ravel(train_y, order='C'))
        #print(grid.score(train_X, train_y))
        #print(grid.score(test_X, test_y))
        bestpar = grid.best_params_
        print("The Best parameters according to grid search are:", bestpar)

        #mod = svm.SVC(kernel='linear', C = bestpar['SVM__C']) #Linear
        mod = svm.SVC(kernel='poly', C = bestpar['SVM__C'], gamma = bestpar['SVM__gamma'], degree= bestpar['SVM__degree']) #Poly
        
        mod.fit(train_X, np.ravel(train_y, order='C'))
        print("Training score for LIBSVM with best parameters:", 100*mod.score(train_X, train_y), "%")
        print("Test score for LIBSVM with best parameters:", 100*mod.score(test_X, test_y), "%")
        ab,a_ind,b_ind=np.intersect1d(train_X, mod.support_vectors_, return_indices=True)
        a_ind = sorted(a_ind)
        boi = []
        for i in range(0, len(a_ind), num_ft):
            boi.append(a_ind[i]//num_ft)
        #print("Indices of support vectors as returned by LIBSVM: ", [temp_train_X.index(x) for x in mod.support_vectors_])
        print("Indices of support vectors as returned by LIBSVM: ", boi)
        #print("Support vectors as returned by LIBSVM:", sorted(mod.support_vectors_, key=lambda x: x[0]))
        print()
        print()
        print()
        
        # CVXOPT STUFF
        print("--------------------------CVXOPT-----------------------------")
        #legr, suppX, suppy, test_pred, acc = cvx_try(train_X, train_y, test_X, test_y, which='linear', C=bestpar['SVM__C']) #Linear
        legr, suppX, suppy, test_pred, acc = cvx_try(train_X, train_y, test_X, test_y, which='poly', C=bestpar['SVM__C'], 
                                                     gamma = bestpar['SVM__gamma'], degree= bestpar['SVM__degree']) #Poly

        print("Test score for CVXOPT with best parameters:", acc, "%")
        ab,a_ind,b_ind=np.intersect1d(train_X, suppX, return_indices=True)
        a_ind = sorted(a_ind)
        boi = []
        for i in range(0, len(a_ind), num_ft):
            boi.append(a_ind[i]//num_ft)
        #print("Indices of support vectors as returned by LIBSVM: ", [temp_train_X.index(x) for x in mod.support_vectors_])
        print("Indices of support vectors as returned by CVXOPT: ", boi)
        #print("Support vectors as returned by CVXOPT:", sorted(suppX, key=lambda x: x[0]))

        
        
        # PLOTTINGS
        #plot_it_all_linear(train_X, train_y, test_X, test_y, reg_pam) #Linear. Nothing for anything else. Can use other one for RBF
        #plot_search_results_linear(grid, reg_pam) #Linear
        plot_search_results(grid) #For non-linear or any with multiple parameters
        
        print()
        print()
        print()
        print()
        print()
        print()

#################################################################################
Labels: 8 , 9
Number of features: 10
#################################################################################
Number of training examples: (454, 10) (454, 1)
Number of test examples: (114, 10) (114, 1)
--------------------------LIBSVM-----------------------------


## RBF

In [ ]:
file_name = '2019EE10143.csv'
split_frac = 0.8
random_state = 69420
low_reg, high_reg = -3,3
low_cof, high_cof = -3,3
low_deg, high_deg = 1, 6

reg_pam = np.logspace(low_reg, high_reg, num=1+high_reg-low_reg)
ker_coeff_pam = np.logspace(low_cof, high_cof, num=1+high_cof-low_cof)
deg_pam = np.linspace(low_deg, high_deg, 1+high_deg-low_deg)


df = pd.read_csv(file_name, header=None)
pairs = [(0,1),(4,6),(8,9)]
features = [10, 25]

for (lab1, lab2) in pairs:
    for num_ft in features:
        print("#################################################################################")
        print("Labels:",lab1, ",", lab2)
        print("Number of features:", num_ft)
        print("#################################################################################")

        #CONVERT TO USE
        df_temp = df.loc[df[25].isin([lab1, lab2])]
        #print(len(df_temp))
        df_temp.iloc[df_temp[25] == lab1, 25] = -1
        df_temp.iloc[df_temp[25] == lab2, 25] = 1
        df_temp = df_temp.sample(frac=1., random_state=random_state)
        
        # SPLIT IN TRAIN AND TEST
        train_df = df_temp[:int(split_frac*len(df_temp))]
        test_df = df_temp[int(split_frac*len(df_temp)):]
        
        # SPLIT BY FEATURES
        X_train_temp = train_df.loc[:, [i for i in range(num_ft)]]
        y_train_temp = train_df.loc[:, [25]]
        X_test_temp = test_df.loc[:, [i for i in range(num_ft)]]
        y_test_temp = test_df.loc[:, [25]]



        train_X = np.array(X_train_temp.values)
        train_y = np.array(y_train_temp.values)
        test_X = np.array(X_test_temp.values)
        test_y = np.array(y_test_temp.values)

        print ("Number of training examples:", train_X.shape, train_y.shape)
        print ("Number of test examples:", test_X.shape, test_y.shape)


        # SVM STUFF & GRID SEARCH CV
        print("--------------------------LIBSVM-----------------------------")
        ppl = [('scaler', StandardScaler()), ('SVM', svm.SVC(kernel='rbf'))]
        pipeline = Pipeline(ppl) 
        #parameters = {'SVM__C':reg_pam} #Linear
        #parameters = {'SVM__C':reg_pam, 'SVM__degree':deg_pam, 'SVM__gamma':ker_coeff_pam} #Poly
        parameters = {'SVM__C':reg_pam, 'SVM__gamma':ker_coeff_pam} #RBF
        grid = GridSearchCV(pipeline, param_grid=parameters, cv=5, return_train_score=True, verbose=0)

        grid.fit(train_X, np.ravel(train_y, order='C'))
        #print(grid.score(train_X, train_y))
        #print(grid.score(test_X, test_y))
        bestpar = grid.best_params_
        print("The Best parameters according to grid search are:", bestpar)

        #mod = svm.SVC(kernel='linear', C = bestpar['SVM__C']) #Linear
        #mod = svm.SVC(kernel='poly', C = bestpar['SVM__C'], gamma = bestpar['SVM__gamma'], degree= bestpar['SVM__degree']) #Poly
        mod = svm.SVC(kernel='rbf', C = bestpar['SVM__C'], gamma = bestpar['SVM__gamma']) #RBF
        
        mod.fit(train_X, np.ravel(train_y, order='C'))
        print("Training score for LIBSVM with best parameters:", 100*mod.score(train_X, train_y), "%")
        print("Test score for LIBSVM with best parameters:", 100*mod.score(test_X, test_y), "%")
        ab,a_ind,b_ind=np.intersect1d(train_X, mod.support_vectors_, return_indices=True)
        a_ind = sorted(a_ind)
        boi = []
        for i in range(0, len(a_ind), num_ft):
            boi.append(a_ind[i]//num_ft)
        #print("Indices of support vectors as returned by LIBSVM: ", [temp_train_X.index(x) for x in mod.support_vectors_])
        print("Indices of support vectors as returned by LIBSVM: ", boi)
        #print("Support vectors as returned by LIBSVM:", sorted(mod.support_vectors_, key=lambda x: x[0]))
        print()
        print()
        print()
        
        # CVXOPT STUFF
        print("--------------------------CVXOPT-----------------------------")
        #legr, suppX, suppy, test_pred, acc = cvx_try(train_X, train_y, test_X, test_y, which='linear', C=bestpar['SVM__C']) #Linear
        #legr, suppX, suppy, test_pred, acc = cvx_try(train_X, train_y, test_X, test_y, which='rbf', C=bestpar['SVM__C'], 
        #                                             gamma = bestpar['SVM__gamma'], degree= bestpar['SVM__degree']) #Poly
        legr, suppX, suppy, test_pred, acc = cvx_try(train_X, train_y, test_X, test_y, which='rbf', C=bestpar['SVM__C'], gamma = bestpar['SVM__gamma']) #RBF

        print("Test score for CVXOPT with best parameters:", acc, "%")
        ab,a_ind,b_ind=np.intersect1d(train_X, suppX, return_indices=True)
        a_ind = sorted(a_ind)
        boi = []
        for i in range(0, len(a_ind), num_ft):
            boi.append(a_ind[i]//num_ft)
        #print("Indices of support vectors as returned by LIBSVM: ", [temp_train_X.index(x) for x in mod.support_vectors_])
        print("Indices of support vectors as returned by CVXOPT: ", boi)
        #print("Support vectors as returned by CVXOPT:", sorted(suppX, key=lambda x: x[0]))

        
        
        # PLOTTINGS
        #plot_it_all_linear(train_X, train_y, test_X, test_y, reg_pam) #Linear. Nothing for anything else. Can use other one for RBF
        plot_it_all_rbf(train_X, train_y, test_X, test_y, reg_pam, ker_coeff_pam)
        #plot_search_results_linear(grid, reg_pam) #Linear
        plot_search_results(grid) #For non-linear or any with multiple parameters
        
        
        print()
        print()
        print()
        print()
        print()
        print()